# ADA Project : Milestone 2

## Cracking the Hollywood interview

On average, the production of a major box office movie costs $65 million, without counting the marketing and distribution fees. Unlike house construction, which usually ends up exactly like the pre-sketched plan, making a movie is unpredictable and anticipating the audience’s opinion is nearly impossible despite the effort and money spent.
Thus, producing it the right way is a crucial job that requires long-time studies and decision-making about the relevant parts that define the movie. This includes the storyline, the script, the actors, the budget, and many more.
This motivates our goal of studying the successful as well as failed films in terms of public ratings collected through the IMDB databases as well as the box office revenue. We mainly analyze the different metrics that define a movie in order to come up with a set of criteria that, if present, will more likely make it successful.

### 1) Imports

In [ ]:
import math
import json
import nltk
import gzip
import pickle
import requests

import numpy as np
import pandas as pd
import seaborn as sns

import scipy.stats as stats
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from collections import Counter
from __future__ import annotations

from nltk.sentiment import SentimentIntensityAnalyzer
import patsy.builtins as pat

import xml.etree.ElementTree as ET
import statsmodels.formula.api as smf

import statsmodels.formula.api as smf
from IPython.display import display, Markdown, Latex



### 2) Utility functions

Some basic functions that may be helpful later.

In [ ]:
def freebase_to_wiki_id(freebase_id: str) -> str | None:
    """
    Query Wikidata ID from Freebase ID
    :param freebase_id: str
    :return: corresponding wiki ID
    """
    url = f"https://www.wikidata.org/w/index.php?search={freebase_id}&title=Special%3ASearch&go=Go"
    res = BeautifulSoup(requests.get(url).text, "html.parser")\
        .find("div", {"class": "mw-search-result-heading"})
    if res is not None:
        return res.find("a")["href"].split("/")[-1]
    return None

path_corenlp = '../data/corenlp_plot_summaries/'

def parse_summaries(file: str) -> BeautifulSoup:
    """
    Parse XML npl processed summary
    :param file: name of
    :return: BeautifulSoup object
    """
    with gzip.open(path_corenlp + file, 'rb') as f:
        data = f.read()
    return BeautifulSoup(data, features="xml")

### 3) Movie metadata analysis and processing

We start by analyzing and cleaning the movies' metadata file ``movie.metadata.tsv``.

#### 3.1) Getting more revenue data

In [ ]:
# Load the dataset
metadata_df = pd.read_csv('data/movie.metadata.tsv', sep="\t")
metadata_df

In [ ]:
# Get some insights
metadata_df.info()

Since the revenue is an important feature in our study, let's see if we have enough data:

In [ ]:
metadata_df['Movie box office revenue'].isna().sum() / metadata_df.shape[0] * 100

Around 90% of the movie revenues are absent. We used the ``boxoffice.csv`` dataset to enrich our dataset:

In [ ]:
def extract_release_year(df):
    """
    Exctracts the release year from the release date as a new column
    :param df: the dataframe
    :return: The extended dataframe
    """
    df['Movie release year'] = df['Movie release date'].astype(str).str[:4]
    df = df[df["Movie release year"].str.contains("nan") == False].copy()
    df['Movie release year'] = df['Movie release year'].apply(lambda x: int(x))
    return df

In [ ]:
# Add a release year column for convenience
metadata_df = extract_release_year(metadata_df)

In [ ]:
# Load the revenue dataset
revenue = pd.read_csv('data/boxoffice.csv', sep=',')

In [ ]:
# Fill some of the absent revenue fields from the boxoffice dataset
metadata_df.set_index(['Movie name','Movie release year'])
revenue.set_index(['title','year'])
metadata_df['Movie box office revenue'].fillna(revenue['lifetime_gross'], inplace=True)

In [ ]:
metadata_df['Movie box office revenue'].isna().sum() / metadata_df.shape[0] * 100

We gained around 20% of relevant data!

#### 3.2) Additional data on movies' ratings

We add information about movie's ratings thanks to the IMDb dataset.
The IMDb dataset is described at <https://www.imdb.com/interfaces/> , which is a version granted by IMDb for academic purposes. We have asked and being authorised by the IMDb team to have access to it.

The following gets IMDb movies' ratings from ``imdb_title_ratings.tsv``.

**IMDb ratings dataset:**

- **tconst**: unique identifier of the movie
- **averageRating**: average of user ratings
- **numVotes**: number of ratings submitted for the movie

In [ ]:
# IMDb ratings dataset
imdb_ratings_df = pd.read_csv('data/imdb_title_ratings.tsv', sep='	')
imdb_ratings_df.head()

**IMDb videos' metadata dataset:**

- **tconst**: unique identifier of title
- **titleType**: type of the title (movie, TV series, etc.)
- **primaryTitle**: title the film is known for
    - **originalTitle**: original title (in original language) of film
- **isAdult**: indicator about adult category
- **startYear**: release year
- **endYear**: end year (for series)
- **runtimeMinutes**: duration of the title in minutes
- **genres**: up to three genres associated to the title

In [ ]:
# IMDb videos' metadata dataset
imdb_names_df = pd.read_csv('data/imdb_title_basics.tsv', sep='	', low_memory=False)
# Only keep audiovisual products labeled as movies
imdb_names_df = imdb_names_df[imdb_names_df['titleType'] == 'movie']
imdb_names_df.head()

In [ ]:
# Join between IMDb movies' rating and metadata
imdb_ratings_meta = imdb_ratings_df.merge(imdb_names_df, on='tconst', how='inner')
imdb_ratings_meta.head()

We merge the original CMU dataset with the IMDb dataset.

In [ ]:
# Join between CMU and IMDb datasets
ratings_merge = metadata_df.merge(imdb_ratings_meta, left_on='Movie name', right_on = 'primaryTitle', how='inner')
ratings_merge.head()

We clean the joined datasets by keeping entries that are consistent in their release dates. In other words, release year has to be similar in both datasets, otherwise we consider an entry may contain erroneous or noisy data and we discard it.

In [ ]:
# Format movie release date
ratings_merge = extract_release_year(ratings_merge)

In [ ]:
# Filter pairs (obtained by join) by almost same release year (max. difference of two)
ratings_merge = ratings_merge[ratings_merge['startYear'] != '\\N']
ratings_merge['releaseDiff'] = (ratings_merge['Movie release year'].astype(int) - ratings_merge['startYear'].astype(int)).abs()
ratings_merge = ratings_merge[ratings_merge['releaseDiff'] <= 2]

We also clean the joined datasets with respect to the movie's runtime.

In [ ]:
# Filter pairs (obtained by join) by almost same runtime (max. difference of 10% with respect of the CMU duration)
# First, we handle the problem of non-existing runtime information for some entries
ratings_merge = ratings_merge[ratings_merge['runtimeMinutes'] != '\\N']
ratings_merge['Movie runtime'].fillna(-1, inplace=True)

ratings_merge['runtimeDiff'] = (ratings_merge['Movie runtime'].astype(int) - ratings_merge['runtimeMinutes'].astype(int)).abs()
ratings_merge = ratings_merge[ratings_merge['runtimeDiff'] <= ratings_merge['Movie runtime']/10]

In [ ]:
ratings_merge.head()

We clean the 'ratings_merge' dataframe (which is the result of joining IMDb's ratings and CMU datasets).
We remove duplicated or non-relevant columns and we order columns in more logical order (with adjacent columns containing similar information)

In [ ]:
# Remove unnecessary columns
ratings_merge = ratings_merge.drop('releaseDiff', axis=1)
ratings_merge = ratings_merge.drop('runtimeDiff', axis=1)
ratings_merge = ratings_merge.drop('endYear', axis=1)
ratings_merge = ratings_merge.drop('Movie name', axis=1)
ratings_merge = ratings_merge.drop('Movie release date', axis=1)
ratings_merge = ratings_merge.drop('runtimeMinutes', axis=1)
ratings_merge = ratings_merge.drop('titleType', axis=1)

# Reorder columns
ratings_merge = ratings_merge[['Wikipedia movie ID', 'Freebase movie ID', 'tconst', 'primaryTitle', 'originalTitle', 'Movie box office revenue', 'averageRating', 'numVotes', 'Movie runtime', 'Movie languages', 'Movie countries (Freebase ID:name tuples)', 'Movie genres (Freebase ID:name tuples)', 'genres', 'isAdult', 'startYear']]

In [ ]:
ratings_merge.head()

Now that 'ratings_merge' represent the an extended 'metadata_df' dataframe (with rating information and more consistent entries), we refer this 'ratings_merge' as the new 'metadata_df'.

In [ ]:
# Assignment for extended dataframe
metadata_df = ratings_merge

# Useful labels for revenue and ratings
box_office_rev = 'Movie box office revenue'
box_office_rat = 'averageRating'

Now, we have a dataframe complete with non null revenues and ratings for sufficiently many movies, ready for use:

In [ ]:
metadata_df = metadata_df[metadata_df[box_office_rev].notnull() & metadata_df[box_office_rat].notnull()].copy()
metadata_df

IMDb genres processing:

In [ ]:
metadata_df["Movie genres: values"] = metadata_df["genres"].apply(lambda x: x.split(","))
metadata_df.drop("genres", inplace=True)

#### 3.3) Initial revenue and rating distributions

In this section, we examine basic facts and distributions of our data, starting with the global distribution of the revenue.

In [ ]:
plt.figure(figsize=(5, 5))
plt.xlabel('Box office revenue', fontsize = 15)
plt.ylabel('Number of movies ', fontsize=15)
plt.title('Box office revenue of all the movies', fontsize=15)
sns.histplot(data=metadata_df[box_office_rev], log_scale=True)
plt.show()

Revenue seems to be slightly heavy tailed. Indeed:

In [ ]:
metadata_df[box_office_rev].describe()

The mean is 3.528042e+07, whilst the median is 5.656388e+06, an order of magnitude smaller.
Let's examine the ratings' distribution:

In [ ]:
plt.figure(figsize=(5, 5))
plt.xlabel('Movie rating', fontsize = 15)
plt.ylabel('Number of movies ', fontsize=15)
plt.title('IMDb rating of all the movies', fontsize=15)
sns.histplot(data=metadata_df[box_office_rat])
plt.show()

This distribution seems closer to normal, with a few peaks.

In [ ]:
metadata_df[box_office_rat].describe()

Median and mean are close. Is the distribution really approximately normal?

In [ ]:
stats.normaltest(metadata_df[box_office_rat])

p-value is basically 0, we can confidently reject the hypothesis that the data comes from a normal distribution.

Let's have a quick look at the influence of the year when a movie was released on both rating and revenue.

In [ ]:
sums_year = metadata_df.groupby('startYear')[[box_office_rev]].sum()[1:]
means_year = metadata_df.sort_values(by="startYear")

In [ ]:
def plot_by_year(df: pd.Series | pd.DataFrame, prefix: str, metric: str, col: str, log_yscale=True):
    """
    Plot a year grouped column
    :param df: said column
    :param prefix: str
    :param metric: what is being plotted
    :param log_yscale: bool
    :param col: where to find data in df
    """
    fig, ax = plt.subplots()

    fig.set_size_inches(16, 8)
    ax.set_xlabel('Release year', fontsize=18)
    ax.set_ylabel(f'{prefix} {metric}', fontsize=16)
    ax.set_title(f'{prefix} {metric} by year', fontsize=16)
    ax.set_label("Floats")
    if prefix == "Mean":
        l = sns.lineplot(data=df, x="startYear", y=col, legend="brief")
    else:
        l = sns.lineplot(data=df, legend="brief")
    if log_yscale:
        l.set_yscale("log")
    plt.xticks(rotation=90)

    plt.show()

In [ ]:
plot_by_year(means_year[80:-10], "Mean", "box office revenue", "Movie box office revenue")

An upwards trend of mean expected revenue is to be expected, especially since more money gets thrown into the film industry:

In [ ]:
plot_by_year(sums_year, "Total", "box office revenue", "Movie box office revenue")

Now let us do the same for rating:

In [ ]:
plot_by_year(means_year[80:-10], "Mean", "rating", "averageRating", log_yscale=False)

Interestingly, between the 1960 and the 1985, IMDb average ratings have dropped, before plateauing. We will have to see if this is tied to the number of ratings submitted. Now, what about the dependence between rating and revenue?

In [ ]:
stats.spearmanr(metadata_df[box_office_rev], metadata_df[box_office_rat])

The p-value being this small rejects that rating and revenue are uncorrelated. However, that correlation seems to be small. Later, it might be interesting to have a look at this correlation *per year* rather than globally.

Lastly, we also plot the runtime just to check whether there is anything interesting to be seen:

In [ ]:
sns.histplot(data=metadata_df["Movie runtime"])

In [ ]:
metadata_df["Movie runtime"].describe()

In [ ]:
plot_by_year(means_year, "Mean", "movie runtime", "Movie runtime", log_yscale=False)

Runtime has increased during the first half of the 20th century. It might be worth studying how runtime affects rating and revenue.

#### 3.4) Effect of genre, country and language on rating and revenue

We move on to the meat and potatoes of metadata analysis: how genre, country and language affect rating and revenue. Some preprocessing first to add indicator variables for each genre, country and language.

In [ ]:
def remove_first_and_last_chars(input_str: str) -> str:
    """
    Remove the first and last char of a string.
    :param input_str
    :return: processed string
    """
    if len(input_str) < 2:
        return ""
    return input_str[1:-1]

def separate_id_from_data(paired_string: str, remove_brackets=True) -> (str, str):
    """
    Separates "{"FreebaseID": "some string"}" strings into ("FreebaseID", "some string") tuples.
    :param paired_string: input paired string
    :param remove_brackets: if True, remove the leading and trailing curly brackets
    :return: said tuple of strings
    """
    if remove_brackets:
        paired_string = remove_first_and_last_chars(paired_string)

    ls = paired_string.split(":")
    for i, s in enumerate(ls):
        ls[i] = remove_first_and_last_chars(s.strip())

    if len(ls) < 2:
        return None, None
    return ls[0], ls[1]

def separate_ids_from_list_data(list_paired_string: str) -> (list, list):
    """
    Separates "{"FreebaseID1": "some string 1", "FreebaseID2": "some string 2", etc.}" strings
    into two lists: (["FreebaseID1", "FreebaseID2", ...], ["some string 1", "some string 2", ...]).
    :param list_paired_string: input list of pairs as string
    :return: said tuple of lists
    """
    list_paired_string = remove_first_and_last_chars(list_paired_string)
    split_pairs = list_paired_string.split(",")
    tupled_pairs = [separate_id_from_data(pair, remove_brackets=False) for pair in split_pairs]
    return [p[0] for p in tupled_pairs], [p[1] for p in tupled_pairs]

def col_to_col_values(column_name: str) -> str:
    """
    String formatting for value column names
    :param column_name: original column name
    :return: said formatted string
    """
    return f"{column_name}: values"

def append_processed_columns(df: pd.DataFrame, column_name: str):
    """
    Separate Freebase IDs from values
    :param df: data, modified in place
    :param column_name: name of column where to separate {Freebase ID: value} pairs
    """
    vals = df[column_name].apply(separate_ids_from_list_data).values.copy()
    df[f"{column_name}: Freebase IDs"] = [vals[i][0] for i in range(len(vals))]
    df[col_to_col_values(column_name)] = [vals[i][1] for i in range(len(vals))]

In [ ]:
metadata_df.rename(columns={"Movie countries (Freebase ID:name tuples)": "Movie countries",
                    "Movie genres (Freebase ID:name tuples)": "Movie genres"}, inplace=True)

cols_to_process = ["Movie countries", "Movie genres", "Movie languages"]
for col in cols_to_process:
    append_processed_columns(metadata_df, col)

metadata_df

In [ ]:
def distinct_values(column_name: str, raw_name: bool=False) -> set:
    """
    Get all values from a column
    :param column_name: said column
    :return: set of values
    """
    col_name = column_name if raw_name else col_to_col_values(column_name)
    return set.union(*metadata_df[col_name].apply(set).values)

def name_appended_column(prefix: str, val: str) -> str:
    """
    Format column name
    :param prefix: str
    :param val: str
    :return: str
    """
    return f"{prefix}: {val}"

def append_indicator_columns(df: pd.DataFrame, all_values: set, column_name: str, prefix: str) -> pd.DataFrame:
    """
    Add columns to the right of a dataframe indicating whether a particular value is present or not
    in some initial column listing values of the same family
    :param df: data (not modified)
    :param all_values: all possible values
    :param column_name: column to inspect
    :param prefix: str
    :return: Dataframe with added columns
    """
    cols = [df[col_to_col_values(column_name)]
            .apply(lambda x: 1 if val in x else 0)
            .rename(name_appended_column(prefix, val))
            for val in all_values]
    cols.insert(0, df)
    return pd.concat(cols, axis=1)

Store all genres and countries encountered:

In [ ]:
all_countries = distinct_values("Movie countries")
all_genres = distinct_values("Movie genres")
all_languages = distinct_values("Movie languages")

genre_prefix = "genre"
langs_prefix = "lang"
cntry_prefix = "country"

Then, let's check the resulting dataframe:

In [ ]:
for ls in [(all_countries, "Movie countries", cntry_prefix),
            (all_genres, "Movie genres", genre_prefix),
            (all_languages, "Movie languages", langs_prefix)]:
    metadata_df = append_indicator_columns(metadata_df, *ls)

metadata_df

For pragmatic purposes such as visualization, we will only treat, in the following, either the $n$ most frequent genres, or genres that are sufficiently frequent.

In [ ]:
from functools import cmp_to_key

def retrieve_n_most_frequent(df: pd.DataFrame, n: int, all_vals: list[str], prefix: str) -> list:
    """
    Retrieve the n most frequent genres, languages or countries, sorted in descending order
    of frequency
    :param df: data
    :param n: integer, max number of values to retrieve
    :param all_vals: all possible values
    :param prefix: str
    :return: said list
    """
    def comparator(val1, val2):
        mean_val1 = df[name_appended_column(prefix, val1)].mean()
        mean_val2 = df[name_appended_column(prefix, val2)].mean()
        return mean_val1 - mean_val2
    return sorted(all_vals, key=cmp_to_key(comparator), reverse=True)[:n]

def retrieve_frequent(df: pd.DataFrame, all_vals: list, prefix: str, freq_threshold=0.05) -> list:
    """
    Filter the values with a sufficiently high frequency
    :param df: data
    :param all_vals: all possible values
    :param prefix: str
    :param freq_threshold: float
    :return: list of sufficiently frequent values
    """
    return list(
        filter(
            lambda val: df[name_appended_column(prefix, val)].mean() > freq_threshold,
            all_vals
        )
    )

def map_to_col_names(data_names: list, prefix: str) -> list:
    """
    Convert data values into column names
    :param data_names: list of  data values
    :param prefix: str
    :return: list of formatted column names
    """
    f = lambda x: name_appended_column(prefix, x)
    return list(map(f, data_names))

In [ ]:
most_freq_genres = retrieve_n_most_frequent(metadata_df, 35, all_genres, genre_prefix)
suff_freq_genres = retrieve_frequent(metadata_df, all_genres, genre_prefix, freq_threshold=0.03)

most_freq_langs = retrieve_n_most_frequent(metadata_df, 35, all_languages, langs_prefix)
suff_freq_langs = retrieve_frequent(metadata_df, all_languages, langs_prefix, freq_threshold=0.03)

most_freq_cntry = retrieve_n_most_frequent(metadata_df, 35, all_countries, cntry_prefix)
suff_freq_cntry = retrieve_frequent(metadata_df, all_countries, cntry_prefix, freq_threshold=0.03)

Our next step will be to examine genres, countries and languages that have are correlated to rating and revenue, and try to understand which ones optimize either metric.

In [ ]:
def find_correlated_metadata(df: pd.DataFrame, freq_data: list, success_metric: str, prefix: str, sig_level=0.05) -> list:
    """
    Among a list of sufficiently frequent data taken from the metadata dataframe,
    find the values such that they are correlated to a movie's success metric with
    a p-value less than sig-level.
    :param df: input dataframe
    :param freq_data: column names to search in
    :param success_metric: str, name of column in df
    :param prefix: str
    :param sig_level: significance level, defaults to 5%
    :return: described list
    """
    correlated_data = []

    for value in freq_data:
        res = stats.spearmanr(df[success_metric], df[name_appended_column(prefix, value)])
        if res.pvalue < sig_level:
            correlated_data.append(value)
    return correlated_data

In [ ]:
correlated_genres_to_revenue = find_correlated_metadata(metadata_df, suff_freq_genres, box_office_rev, genre_prefix)
correlated_langs_to_revenue = find_correlated_metadata(metadata_df, most_freq_langs, box_office_rev, langs_prefix)
correlated_cntry_to_revenue = find_correlated_metadata(metadata_df, most_freq_cntry, box_office_rev, cntry_prefix)

correlated_genres_to_rating = find_correlated_metadata(metadata_df, suff_freq_genres, box_office_rat, genre_prefix)
correlated_langs_to_rating = find_correlated_metadata(metadata_df, most_freq_langs, box_office_rat, langs_prefix)
correlated_cntry_to_rating = find_correlated_metadata(metadata_df, most_freq_cntry, box_office_rat, cntry_prefix)

In [ ]:
def plot_metadata_frequency_against_metric(df: pd.DataFrame, prefix: str,  titled_data: list, success_metric: str, title: str, log_scale=True):
    """
    Generating a grid of histograms
    :param df: data
    :param prefix: str
    :param titled_data: titles of data, to be converted to column names
    :param success_metric: measured column name
    :param title: str, figure title
    :param log_scale: determines the scale of the axes
    """

    # Making the data fit into a square grid...
    squares = np.arange(8) ** 2
    shifted_squares = squares - len(titled_data)
    smallest_big_enough_square = squares[np.argmax(shifted_squares > 0) - 1]

    tested_data = map_to_col_names(titled_data, prefix)[:smallest_big_enough_square]
    size = int(np.sqrt(smallest_big_enough_square))

    fig, ax = plt.subplots(size, size, figsize=(11, 11), sharex = True)
    for i in range(smallest_big_enough_square):
        sbplt = ax[i%size, math.floor(i/size)]
        sns.histplot(ax=sbplt, data=df[df[tested_data[i]] == 1][success_metric], log_scale=log_scale)
        sbplt.set_title(titled_data[i])

    fig.suptitle(title, fontsize=18)
    fig.tight_layout()

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, genre_prefix, correlated_genres_to_revenue, box_office_rev, title="Plotting revenue against genres")

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, genre_prefix, correlated_genres_to_rating, box_office_rat, title="Plotting rating against genres", log_scale=False)

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, cntry_prefix, correlated_cntry_to_revenue, box_office_rev, title="Plotting revenue against countries")

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, cntry_prefix, correlated_cntry_to_rating, box_office_rat, title="Plotting rating against countries", log_scale=False)

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, langs_prefix, correlated_langs_to_revenue, box_office_rev, title="Plotting revenue against languages")

In [ ]:
plot_metadata_frequency_against_metric(metadata_df, langs_prefix, correlated_langs_to_rating, box_office_rat, title="Plotting rating against languages", log_scale=False)

Interesting differences can already be seen. For example, the 3 initial peaks in the rating histogram have become 5 peaks in English drama movies coming from the US. Later, it will be interesting to check the correlation of rating and revenue on a *per year* basis too.

### 4) Actors Metadata analysis

Here we focus on analysing the characters metadata in ``charcater.metadata.tsv``

In [ ]:
# Load the dataset
names_char = ['Wikipedia movie ID',
              'Freebase movie ID',
              'Movie release date',
              'Character name',
              'Actor date of birth',
              'Actor gender',
              'Actor height',
              'Actor ethnicity',
              'Actor name',
              'Actor age at movie release',
              'Freebase character/actor map ID',
              'Freebase character ID',
              'Freebase actor ID']

characters_metadata_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names = names_char)

In [ ]:
characters_metadata_df.head()

In [ ]:
characters_metadata_df.info()

In [ ]:
characters_metadata_df['Actor gender'].describe()

##### Age at movie release:

In [ ]:
characters_metadata_df['Actor age at movie release'].describe()

We noticed that the attribute 'Age at movie release' contains negative values

In [ ]:
characters_metadata_df[characters_metadata_df['Actor age at movie release'] < 0]

In [ ]:
characters_metadata_df['Actor age at movie release'] = characters_metadata_df['Actor age at movie release'].apply(abs)

In [ ]:
sns.histplot(data=characters_metadata_df, x="Actor age at movie release")
plt.xlim(0, 100)

##### Titles known for:
Since our study focuses on analysing features that contribute to a movie's success, it would be relevant to study actors performance. This can be done by having information on when each actor performed the best.
For this task, we used the  ``imdb_name_basics.tsv`` dataset, which contains the 4 topmost titles an actor is known for.

**IMDb actors dataset:**

- **nconst**: unique identifier of person
- **primaryName**: name the person is known for
- **birthYear**: birth year
- **deathYear**: death year
- **primaryProfession**: top 3 professions of the person
- **knownForTitles**: movies the person is known for


In [ ]:
# Load the IMDb actors dataset
imdb_actors_df = pd.read_csv('data/imdb_name_basics.tsv', sep='	')

In [ ]:
imdb_actors_df.head()

In [ ]:
imdb_actors_df.info()

Since we only have the id of the known for titles, we fetch the title names from the `imdb_names_df`

First we separate each of the titles ids an actor is known for in a single column, then we fetch the name corresponding to each id.

In [ ]:
def get_title_by_index(index):
    """
    Returns the title id at a given index
    :param index: The id index in the list of ids
    :return: The id
    """
    def f(array):
        list_ids = array.split(",")
        corresponding_id = ""
        if index < len(list_ids):
            corresponding_id = list_ids[index]
        return corresponding_id

    return f

imdb_actors_df['knownForTitles1'] = imdb_actors_df['knownForTitles'].apply(get_title_by_index(0))
imdb_actors_df['knownForTitles2'] = imdb_actors_df['knownForTitles'].apply(get_title_by_index(1))
imdb_actors_df['knownForTitles3'] = imdb_actors_df['knownForTitles'].apply(get_title_by_index(2))
imdb_actors_df['knownForTitles4'] = imdb_actors_df['knownForTitles'].apply(get_title_by_index(3))

In [ ]:
imdb_actors_df.head()

In [ ]:
# IMDb movies' metadata dataset
imdb_names_df.head()

In [ ]:
# Keep only relevant columns
imdb_names_reduced_df = imdb_names_df[["tconst", "primaryTitle"]]

In [ ]:
def assign_known_title(index, actors_df, names_df):
    """
    Fetch the known titles names
    :param index: Index of the title in the list
    :param actors_df: actors dataframe
    :param names_df: movie names dataframe
    :return:
    """
    actors_df = actors_df.merge(names_df, left_on="knownForTitles{}".format(index), right_on = 'tconst', how='inner')
    actors_df["knownForTitles{}".format(index)] = actors_df['primaryTitle']
    actors_df = actors_df.drop('tconst', axis=1)
    actors_df = actors_df.drop('primaryTitle', axis=1)
    return actors_df

In [ ]:
imdb_actors_df = assign_known_title(1, imdb_actors_df, imdb_names_reduced_df)
imdb_actors_df = assign_known_title(2, imdb_actors_df, imdb_names_reduced_df)
imdb_actors_df = assign_known_title(3, imdb_actors_df, imdb_names_reduced_df)
imdb_actors_df = assign_known_title(4, imdb_actors_df, imdb_names_reduced_df)

In [ ]:
imdb_actors_df.head()

In [ ]:
imdb_actors_df['knownForTitlesNames'] = imdb_actors_df[imdb_actors_df.columns[6:]].apply(
    lambda x: ','.join(x),
    axis=1
)

In [ ]:
imdb_actors_df = imdb_actors_df.drop('knownForTitles1', axis = 1).drop('knownForTitles2', axis = 1).drop('knownForTitles3', axis = 1).drop('knownForTitles4', axis = 1)

In [ ]:
imdb_actors_df.head()

As an example, let's see what are the titles Leonardo Dicaprio is known for

In [ ]:
result = imdb_actors_df[imdb_actors_df['primaryName'] == "Leonardo DiCaprio"]['knownForTitlesNames']
print(result)

No surprise !

We add the IMDb's casting dataset. It contains the people that have participated in castings for movies and their role in them.
We are not directly using this dataframe in Milestone 2 as it will be joined and applied to data obtained by methods in Milestone 3 (refer to Method 2 at Readme for further information).

**IMDb castings dataset:**

- **tconst**: unique identifier of title
- **ordering**: index identify a row within a specific title
- **nconst**: unique identifier of person
- **category**: category of job in casting
- **job**: job title
- **characters**: name of character if it was an interpretation role

In [92]:
# IMDb castings dataset
imdb_casting_df = pd.read_csv('data/imdb_title_principals.tsv', sep='	', low_memory=False)
imdb_names_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb_title_principals.tsv'

### 5) tvtropes clusters analysis

Here we use the clusters in ``tvtropes.clusters.txt`` to see what type of characters an actor tends to play.

In [ ]:
# Load the data into a dataframe
with open('data/tvtropes.clusters.txt') as f:
    d = []
    for row in f.readlines():
        t, json_data = row.split('\t')
        data = json.loads(json_data)
        data['type'] = t
        d.append(data)

tv_tropes_df = pd.DataFrame(d)
tv_tropes_df.set_index("id", inplace=True)

In [ ]:
tv_tropes_df.head()

In [ ]:
# split the type in tokens
tv_tropes_df['type'] = tv_tropes_df['type'].apply(lambda x: ' '.join(x.split('_')))

In [ ]:
tv_tropes_df['type'].head()

In order to have better insight on the data and also for further analysis on what type of characters are played by each actor, we decide to conduct sentiment analysis on those type. For this task we use the nltk library.

In [ ]:
# Load the model
nltk.download('vader_lexicon')

In [ ]:
# Initialization and example
sia = SentimentIntensityAnalyzer()
sia.polarity_scores('adventurer archeologist')

In [ ]:
def get_sentiment(sentence):
    """
    Compute the sentiment value of a sentence
    :param sentence: The sentence
    :return: -1 for negative, 0 for neutral and 1 for positive
    """
    score = sia.polarity_scores(sentence)
    # If the score is more positive with a less significant neutral value and vice versa
    if score['pos'] > score['neg'] and score['neu'] < 0.5: return 1
    elif score['pos'] < score['neg'] and score['neu'] < 0.5: return -1
    else: return 0

In [ ]:
# Create a column sentiment containing the sentiment value of the character type
tv_tropes_df['sentiment'] = tv_tropes_df['type'].apply(get_sentiment).to_frame()

In [ ]:
tv_tropes_df.head(10)

In [ ]:
import matplotlib.patches as mpatches

tv_tropes_df['sentiment'].value_counts().plot(kind='bar', xlabel= 'sentiment', ylabel = 'count', title = 'count of each sentiment')
neg_patch = mpatches.Patch(label='-1 = negative')
pos_patch = mpatches.Patch(label='1 = positive')
neu_patch = mpatches.Patch(label='0 = neutral')
blue_patch = mpatches.Patch(color='blue', label='Positive')

plt.legend(handles=[neg_patch, pos_patch, neu_patch])

### 6) Plot analysis

Here we do some processing on the plot summaries for further use in Milestone 3.

In [ ]:
# Load the plot summaries
plots_df = pd.read_csv('data/plot_summaries.txt', sep='	')
plots_df = plots_df[plots_df["wikipedia_id"].isin(metadata_df["Wikipedia movie ID"])]

In [ ]:
plots_df.head()

In [ ]:
len(plots_df)

In [ ]:
subpath = "data/corenlp_plot_summaries/"
starting_positions = {"VB", "NN", "NP", "PP", "RB"}
import os

def get_important_lemmas(wiki_id: int) -> list[str]:
    """
    Retrieve important lemmas from
    :param wiki_id: wikipedia movie id
    :return: list of lemmas of important words
    """

    def is_important(token):
        tok = token.find("POS").text
        for pos in starting_positions:
            if tok.startswith(pos):
                return True
        return False

    def to_lemma(token):
        return token.find("lemma").text

    zip_name = str(wiki_id) + ".xml.gz"

    if not os.path.isfile(subpath + zip_name):
        print(f"Missing file with id {wiki_id}")
        return []

    with gzip.open(subpath + zip_name, 'r') as file:
        xmltree = ET.ElementTree(ET.fromstring(file.read())).getroot()

    return list(map(to_lemma, filter(is_important, xmltree.iter("token"))))

In [ ]:
%%time
pickle_path = "data/pickled_data/plots_pickle.pkl"
if os.path.isfile(pickle_path):
    plots_df = pd.read_pickle(pickle_path)
else:
    plots_df['important_lemmas'] = plots_df['wikipedia_id'].apply(get_important_lemmas)
    plots_df.to_pickle(pickle_path)
plots_df.head()

In [ ]:
len(plots_df)

In [ ]:
lengths = plots_df["important_lemmas"].apply(len)
print(f"{len(plots_df[lengths == 0])} movie(s) has/have no core nlp plot summary.")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x)